Generate pages

## Read the XBRL-instance

In [1]:
from arelle import ViewFileDTS, ViewFileFactTable, ViewFileFactList, ModelManager, FileSource, ViewFileRelationshipSet
from arelle import Cntlr, ModelXbrl, ModelDocument, ViewFileFormulae
from arelle import ModelFormulaObject, RenderingEvaluator, XbrlConst
from arelle.ModelDtsObject import ModelRelationship
import arelle

In [2]:
import src
import pandas as pd
import re
import os
from os import listdir, walk, makedirs
from os.path import isfile, join, exists
from io import StringIO

In [3]:
# make sure you have a 'arelle' directory in the data_path! (This is where the taxonomy is stored)
XBRL_DATA_PATH    = 'C:\\Users\\wjwil\\20_local_data\\xbrl\\' 
#DATA_PATH    = 'H:\\20_local_data\\xbrl\\' 
XBRL_RESULTS_PATH = 'C:\\Users\\wjwil\\50_results\\xbrl\\'
LANGUAGE     = "en-GB"
# set the location of taxonomy
os.environ['XDG_CONFIG_HOME'] = XBRL_DATA_PATH 

In [4]:
DOCS_PATH ="..\\docs\\"
DATA_PATH ="..\\data\\"
RULES_PATH = "..\\Solvency2-rules\\"

In [5]:
# Now we make a modelmanager
controller = Cntlr.Cntlr()
controller.webCache.workOffline = True

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

In [6]:
# And we read the XBRL instance
XBRL_INSTANCE = 'qrs_240_instance.xbrl'

xbrl_instance = ModelXbrl.load(modelmanager, XBRL_DATA_PATH + XBRL_INSTANCE)

In [7]:
tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
tables.sort()
print(tables[0:5])

['http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.01.01.02.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.01.02.01.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.02.01.02.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.05.01.02.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.05.01.02.02']


## Extract from XBRL-instance

In [8]:
#ViewFileDTS.viewDTS(xbrl_instance, XBRL_RESULTS_PATH + "dts.csv")

In [10]:
RenderingEvaluator.init(xbrl_instance)
for table in tables:
    src.toDataFrame.generateCSVTables(xbrl_instance, XBRL_RESULTS_PATH, table_uri = table, lang = LANGUAGE)

Closed definition node s2md_c74 does not contribute at least one structural node
Closed definition node s2md_c499 does not contribute at least one structural node
Closed definition node s2md_c2703 does not contribute at least one structural node
Closed definition node s2md_c2707 does not contribute at least one structural node
Closed definition node s2md_c2714 does not contribute at least one structural node
Closed definition node s2md_c4300 does not contribute at least one structural node
Closed definition node s2md_c8902 does not contribute at least one structural node
Closed definition node s2md_c8912 does not contribute at least one structural node
Closed definition node s2md_c8968 does not contribute at least one structural node


In [ ]:
# Generate English table labels
template_dict = {}
linkRoleUris = xbrl_instance.relationshipSet("Table-rendering").linkRoleUris
for role_uri in linkRoleUris:
    definition = os.path.basename(role_uri)
    tblAxisRelSet = xbrl_instance.relationshipSet(XbrlConst.euTableAxis, role_uri)
    if len(tblAxisRelSet.modelRelationships)==0:
        tblAxisRelSet = xbrl_instance.relationshipSet((XbrlConst.tableBreakdown, XbrlConst.tableBreakdownMMDD, XbrlConst.tableBreakdown201305, XbrlConst.tableBreakdown201301, XbrlConst.tableAxis2011), role_uri)
    for rootconcept in tblAxisRelSet.rootConcepts:
        # rootconcept is a modelTable-object
        template_dict[rootconcept.definitionLabelsView[2][1]] = rootconcept.definitionLabelsView[3][1]

In [ ]:
# # Generate dictionary with rc-codes to labels
# tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
# tables.sort()

# datapoint_dict = {}
# RenderingEvaluator.init(xbrl_instance)
# for table in tables:
#     d = src.rc2label.rc2label_dict(xbrl_instance, RESULTS_PATH, table_uri = table, lang = LANGUAGE)
#     datapoint_dict.update(d)

In [ ]:
FILENAME_TAXO = 'simple_taxo.csv'
df_taxo = pd.read_csv(join(DATA_PATH, FILENAME_TAXO))
datapoint_dict = {}
for row in df_taxo.index:
    label = df_taxo.loc[row, 'label']
    datapoint_dict[df_taxo.loc[row, 'datapoint']] = str(label)
    

## List validation rules in taxonomy

In [ ]:
ViewFileFormulae.viewFormulae(xbrl_instance, XBRL_RESULTS_PATH + "formulae.csv", "header", None)

In [ ]:
formulae = pd.read_csv(XBRL_RESULTS_PATH + "formulae.csv")
df_xbrl = formulae[formulae['Expression'].str[0:2]=="BV"]

In [ ]:
len(df_xbrl.index)

In [ ]:
df = pd.DataFrame()
for row in df_xbrl.index:
    expr = df_xbrl.loc[row, 'Expression']
    label = df_xbrl.loc[row, 'Label']
    rule_templates, rule_datapoints, rule_id, rule_ref = src.parse_formula(expr, syntax = "XBRL")
    rule_date = "Unknown"
    df = df.append(pd.DataFrame(data = [[rule_id, label, rule_templates, rule_datapoints, rule_ref, expr]]), ignore_index = True)
df.columns = ['Rule id', 'Rule label', 'Rule templates', 'Rule datapoints', 'Rule references', 'Rule expression']

## Generate subdirectories for XBRL Formulae

In [ ]:
DIR_NAME = "EIOPA Validation rules_2.4.0\\"

all_templates = []
for row in df.index:
    all_templates.extend(df.loc[row, "Rule templates"])
templates = list(pd.Series(data = all_templates).sort_values().unique())

for template in templates:
    if not exists(join(DOCS_PATH, DIR_NAME, template)):
        makedirs(join(DOCS_PATH, DIR_NAME, template))

In [ ]:
src.write_rst(join(DOCS_PATH, DIR_NAME), df, template_dict, datapoint_dict)

## Read DNB Validation rules

In [ ]:
FILE_NAME = '2020-01-22 Set aanvullende controleregels Solvency II_tcm46-387021.xlsx'
DIR_NAME = "DNB Additional validation rules.2.4.0"

df_dnb = pd.read_excel(os.path.join(RULES_PATH, FILE_NAME), header = 1)
df_dnb = df_dnb.set_index('ControleRegelCode')
df_dnb = df_dnb.drop('S.01.03_110', axis = 0)
df_dnb.fillna("", inplace = True)

In [ ]:
df = pd.DataFrame()
for row in df_dnb.index:
    expr = df_dnb.loc[row, 'Formule']
    label = ""
    rule_templates, rule_datapoints, rule_id, rule_ref = src.parse_formula(expr, syntax = "EVA2")
    rule_id = row
    rule_date = "2010-01-22"
    df = df.append(pd.DataFrame(data = [[rule_id, label, rule_templates, rule_datapoints, rule_ref, expr]]), ignore_index = True)
df.columns = ['Rule id', 'Rule label', 'Rule templates', 'Rule datapoints', 'Rule references', 'Rule expression'] 

## Generate subdirectories for DNB Validation rules

In [ ]:
all_templates = []
for row in df.index:
    all_templates.extend(df.loc[row, "Rule templates"])
templates = list(pd.Series(data = all_templates).sort_values().unique())

for template in templates:
    if not exists(join(DOCS_PATH, DIR_NAME, template)):
        makedirs(join(DOCS_PATH, DIR_NAME, template))

In [ ]:
src.write_rst(join(DOCS_PATH, DIR_NAME), df, template_dict, datapoint_dict)